<a href="https://colab.research.google.com/github/jasminl/adaptics/blob/master/onnx_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install onnx
!pip install onnxruntime
!pip install onnxruntime_extensions
!pip install numpy

In [ ]:
from onnx import helper, onnx_pb as onnx_proto, save_model
from onnxruntime_extensions import onnx_op, PyCustomOpDef, make_onnx_model

nodes = [helper.make_node('CustomOp', ['input'], ['output_1'], domain='ai.onnx.contrib'),
         helper.make_node('Abs', ['output_1'], ['output_2'])]

@onnx_op(op_type="CustomOp",outputs=[PyCustomOpDef.dt_float])
def custom_two_op(f):
    print('Arbitrary code')
    return f

input = helper.make_tensor_value_info('input', onnx_proto.TensorProto.FLOAT, [1, 1])
output_1 = helper.make_tensor_value_info('output_1', onnx_proto.TensorProto.FLOAT, [1, 1])
output_2 = helper.make_tensor_value_info('output_2', onnx_proto.TensorProto.FLOAT, [1, 1])

graph = helper.make_graph(nodes, 'test', [input], [output_2])
model = helper.make_model(graph,
                          opset_imports=[helper.make_operatorsetid('ai.onnx.contrib', 1)],
                          ir_version=7)

save_model(model, 'mymodel.onnx')

In [ ]:
from onnx import load_model
import onnxruntime as _ort
from onnxruntime_extensions import (
    onnx_op, PyCustomOpDef, make_onnx_model,
    get_library_path as _get_library_path)
import numpy as np

so = _ort.SessionOptions()

# Define a custom operator: this must be defined before the call to register_custom_ops_library
# @onnx_op(op_type="CustomOp",outputs=[PyCustomOpDef.dt_float])
# def custom_two_op(f):
#     print('Arbitrary code')
#     return np.round(f)

so.register_custom_ops_library(_get_library_path())  # Must first call any @onnx_op before this

new_model = load_model('mymodel.onnx')

sess = _ort.InferenceSession(new_model.SerializeToString(), so)
res = sess.run(None, {'input': np.random.rand(1, 1).astype(np.float32)})

In [ ]:
import json
import io
import onnx
from onnx import load_model

new_model = load_model('mymodel.onnx')

# Do model checks (if greater than 2GB, use file path instead:
# https://github.com/onnx/onnx/blob/main/docs/PythonAPIOverview.md)
try:
    onnx.checker.check_model(new_model)
except onnx.checker.ValidationError as e:
    print(f"The model is invalid: {e}")
else:
    print("The model is valid!")

# Check for presence of custom operators (even if implementation is lacking)
for node in new_model.graph.node:
    if node.domain != '' and node.domain != 'ai.onnx':
        print('Custom operator found')
        print(node)
